使用朴素贝叶斯构建垃圾邮件过滤器

在这个项目中，我们将使用多项朴素贝叶斯算法为SMS消息构建垃圾邮件过滤器。 我们的目标是编写一个对新邮件进行分类的程序，其准确性大于80％，因此我们希望将80％以上的新邮件正确分类为垃圾邮件或垃圾邮件（非垃圾邮件）。

为了训练该算法，我们将使用5572条已被人类分类的SMS消息的数据集。 该数据集由Tiago A. Almeida和JoséMaríaGómezHidalgo组合而成，可以从UCI机器学习存储库中下载。 本页上更详细地描述了数据收集过程，您还可以在其中找到Tiago A. Almeida和JoséMaríaGómezHidalgo撰写的一些论文。

探索数据集
现在，我们将从读取数据集中开始。

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('E:/学习资料/python数据分析/dataquest/第五单元/项目四/')

import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
sms_spam = pd.read_csv('SMSSpamCollection',sep='\t',header=None,names=['Label', 'SMS'])
print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


在下面，我们看到大约87％的邮件是非垃圾邮件，其余13％是垃圾邮件。 该示例看起来很有代表性，因为在实践中，人们收到的大多数邮件都是非垃圾邮件。

In [3]:
sms_spam['Label'].value_counts(normalize = True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

培训和测试集

现在，我们将数据集分为训练和测试集，其中训练集占数据的80％，测试集占其余20％。

In [4]:
# Randomize the dataset
data_randomized = sms_spam.sample(frac = 1,random_state = 1)

# Calculate index for split
training_test_index = round(len(data_randomized)*0.8)
training_set = data_randomized[:training_test_index].reset_index(drop = True)
test_set = data_randomized[training_test_index:].reset_index(drop = True)

# Training/Test split
print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


现在，我们将分析培训和测试集中的垃圾邮件和非垃圾邮件的百分比。 我们预计百分比将接近完整数据集中的百分比，其中约有87％的邮件为垃圾邮件，其余13％为垃圾邮件。

In [5]:
training_set['Label'].value_counts(normalize = True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [6]:
test_set['Label'].value_counts(normalize = True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

数据清理
要计算算法所需的所有概率，我们首先需要执行一些数据清除操作，以使数据具有某种格式，从而使我们能够轻松提取所需的所有信息。

本质上，我们希望将数据采用分隔的形式（将语句拆分成词语）





字母大小写和标点符号

我们将首先删除所有标点符号，然后将每个字母都转换为小写字母。

In [7]:
# After cleaning
training_set['SMS'] = training_set['SMS'].str.replace('\W',' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


创建词汇

现在让我们开始创建词汇表，在这种情况下，这意味着列表中包含训练集中的所有唯一单词。

In [8]:
training_set['SMS'] = training_set['SMS'].str.split()
vocabulary = []
for lst in training_set['SMS']:
    for i in lst:
        vocabulary.append(i)

vocabulary = set(vocabulary)
vocabulary = list(vocabulary)

In [9]:
len(vocabulary)

7783

最终训练集

现在，我们将使用刚刚创建的词汇表进行所需的数据转换。

In [10]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

In [11]:
for index,sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [12]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [13]:
training_set_clean = pd.concat([training_set,word_counts],axis = 1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


首先计算常数
现在，我们已经完成了训练集的清理工作，可以开始创建垃圾邮件过滤器了。 朴素贝叶斯算法将需要回答以下两个概率问题，才能对新消息进行分类：


P(Spam | w_1,w_2, ..., w_n)-- P(Spam)*P(w_i|Spam)

P(Ham | w_1,w_2, ..., w_n)-- P(Ham)*P(w_i|Ham)


另外，要在上述公式中计算P（wi | Spam）和P（wi | Ham），我们需要使用以下公式：


P(w_i|Spam) =(N_{w_i|Spam}+ alpha) / N_{Spam} + alpha*N_{Vocabulary}}

P(w_i|Ham) = ({N_{w_i|Ham} + alpha / {N_{Ham} + alpha*N_{Vocabulary}}


上面四个等式中的某些术语对于每个新消息都具有相同的值。 我们可以一次计算这些项的值，并避免在收到新消息时再次进行计算。下面，我们将使用训练集来计算：

P（垃圾邮件）和P（垃圾邮件）

NSpam，NHam，NV词汇

我们还将使用拉普拉斯平滑并设置alpha = 1

In [14]:
# Isolating spam and ham messages first
spam_message = training_set_clean[training_set_clean['Label'] == 'spam']
ham_message = training_set_clean[training_set_clean['Label'] == 'ham']


# P(Spam) and P(Ham)
p_spam = len(spam_message) / len(training_set_clean)
p_ham = len(ham_message) / len(training_set_clean)

# N_Spam and N_ham
n_spam = spam_message['SMS'].apply(len).sum()
n_ham = ham_message['SMS'].apply(len).sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

print(p_spam)
print(p_ham)
print(n_spam)
print(n_ham)
print(n_vocabulary)

0.13458950201884254
0.8654104979811574
15190
57237
7783


计算参数
现在我们有了上面计算的常数项，我们可以继续计算参数$ P（w_i | Spam）$和$ P（w_i | Ham）$。 因此，每个参数将是与词汇表中每个单词相关联的条件概率值。

使用以下公式计算参数：


P(wi|Spam) =(N{wi|Spam}+ alpha) / N{Spam} + alpha*N_{Vocabulary}}

P(wi|Ham) = ({N{wi|Ham} + alpha / {N{Ham} + alpha*N_{Vocabulary}}




In [15]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_message[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_message[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

分类新消息
现在我们已经计算了所有参数，我们可以开始创建垃圾邮件过滤器了。 垃圾邮件过滤器可以理解为以下功能：

输入新消息（w1，w2，...，wn）作为输入。

计算P（Spam | w1，w2，...，wn）和P（Ham | w1，w2，...，wn）。

比较P（Spam | w1，w2，...，wn）和P（Ham | w1，w2，...，wn）的值，并：

如果P（Ham | w1，w2，...，wn）> P（Spam | w1，w2，...，wn），则邮件被分类为ham。

如果P（Ham | w1，w2，...，wn）<P（Spam | w1，w2，...，wn），则邮件被归类为垃圾邮件。

如果P（Ham | w1，w2，...，wn）= P（Spam | w1，w2，...，wn），则该算法可能需要人工帮助。


In [21]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message  *= parameters_ham[word]
        
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [22]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: spam


In [23]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: ham


测量垃圾邮件过滤器的准确性

上面的两个结果看起来很有希望，但让我们看看过滤器在测试集上的表现如何，测试集有1,114条消息。

我们将从编写一个返回分类标签而不是打印它们的函数开始。

In [26]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message  *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'need human classification'
    
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)

In [27]:
correct = 0
total = len(test_set)

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)     

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


准确率接近98.74％，非常好。 我们的垃圾邮件过滤器查看了1,114条在培训中未发现的邮件，并正确分类了1,100条。

下一步
在此项目中，我们设法使用多项朴素贝叶斯算法为SMS消息构建了垃圾邮件过滤器。 该滤波器在我们使用的测试装置上的精度为98.74％，这是一个相当不错的结果。 我们最初的目标是达到80％以上的准确性，并且我们设法做到的更好。

后续步骤包括：

分析错误分类的14条消息，并尝试找出为什么算法对它们进行错误分类的原因
通过使算法对字母大小写敏感，使过滤过程更加复杂